# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

메모리는 에이전트 호출간에 지속되는 메시지를 말한다. 즉 에이전트의 상태(state)를 뜻한다.

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv("../.env")

# Check and print results
doublecheck_env("../.env")

OPENAI_API_KEY=****VLwA
UPSTAGE_API_KEY=****d51g
LANGCHAIN_TRACING_V2=false
LANGCHAIN_ENDPOINT=****.com
LANGCHAIN_PROJECT=****demy
LANGCHAIN_API_KEY=****fc10
PINECONE_API_KEY=****4e73
HUGGINGFACE_API_KEY=****aRMB
TAVILY_API_KEY=****2UUC
SERPER_API_KEY=****e4cd
WOLFRAM_ALPHA_APPID=****LT74
POLYGON_API_KEY=****8Gso


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

# - 단계적으로 생각하세요.
# - 데이터가 필요할 때는 `execute_sql` 도구를 하나의 SELECT 쿼리로 호출하세요.
# - 읽기 전용만 허용; INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE는 금지합니다.
# - 사용자가 명시적으로 달리 요청하지 않는 한 5행으로 제한하세요.
# - 도구가 'Error:'를 반환하면 SQL을 수정하고 다시 시도하세요.
# - 명시적인 열 목록을 선호하십시오; SELECT *는 피하십시오.

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [ ]:
question = "저는 Frank Harris, 최근 invoce의 total이 얼마였나요?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step) # 모든 단계를 기록하기 위해.(디버깅, 로깅 위해)

================================ Human Message =================================

저는 Frank Harris, 최근 invoce의 total이 얼마였나요?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_VRezWRWLvAP5X0ZkwIO9Fl1g)
 Call ID: call_VRezWRWLvAP5X0ZkwIO9Fl1g
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Invoice AS i
JOIN Customer AS c ON c.CustomerId = i.CustomerId
WHERE c.FirstName = 'Frank' AND c.LastName = 'Harris'
ORDER BY i.InvoiceDate DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]
================================== Ai Message ==================================

Frank Harris님의 가장 최근 송장 총액은 $5.94입니다. (송장 ID: 374, 날짜: 2013-07-04)


In [8]:
question = "title들이 뭐였죠?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

title들이 뭐였죠?
================================== Ai Message ==================================

어느 테이블(혹은 어떤 데이터)의 title 값을 말씀하시는 건가요? 테이블명을 알려주시면 해당 테이블의 title을 최대 5개까지 보여드릴게요.

원하시면 ‘title’ 컬럼이 있는 테이블 후보를 몇 개 찾아서 먼저 알려드릴 수도 있어요.


## Add memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

In [10]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [11]:
question = "저는 Frank Harris입니다. 최근 invoce의 total이 얼마였나요?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

저는 Frank Harris입니다. 최근 invoce의 total이 얼마였나요?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_lfvQZ5Fe02GSZ7OhrmgLnDo3)
 Call ID: call_lfvQZ5Fe02GSZ7OhrmgLnDo3
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Invoice AS i
JOIN Customer AS c ON c.CustomerId = i.CustomerId
WHERE c.FirstName = 'Frank' AND c.LastName = 'Harris'
ORDER BY datetime(i.InvoiceDate) DESC, i.InvoiceId DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]
================================== Ai Message ==================================

최근 인보이스의 총액은 $5.94입니다. (인보이스 날짜: 2013-07-04)


In [12]:
question = "title들이 뭐였죠?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

title들이 뭐였죠?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_10sOBOrUKyPZuNLT9HVF15Lm)
 Call ID: call_10sOBOrUKyPZuNLT9HVF15Lm
  Args:
    query: SELECT t.Name AS Title
FROM InvoiceLine AS il
JOIN Track AS t ON t.TrackId = il.TrackId
WHERE il.InvoiceId = (
  SELECT i.InvoiceId
  FROM Invoice AS i
  JOIN Customer AS c ON c.CustomerId = i.CustomerId
  WHERE c.FirstName = 'Frank' AND c.LastName = 'Harris'
  ORDER BY datetime(i.InvoiceDate) DESC, i.InvoiceId DESC
  LIMIT 1
)
ORDER BY il.InvoiceLineId ASC
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',)]
================================== Ai Message ==================================

최근 인보이스에 포함된 트랙 제목(최대 5개):
- Holier Than Thou
- Th

## Try your own queries
Now that there is memory, check the agents recall!

In [ ]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)